ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytket-qiskit 0.36.0 requires qiskit~=0.41.0, but you have qiskit 0.42.1 which is incompatible.
pytket-qiskit 0.36.0 requires qiskit-aer~=0.11.2, but you have qiskit-aer 0.12.0 which is incompatible.
pytket-qiskit 0.36.0 requires qiskit-ibm-runtime~=0.8.0, but you have qiskit-ibm-runtime 0.9.3 which is incompatible.


In [ ]:
from pytket.extensions.qiskit import set_ibmq_config
set_ibmq_config(hub='ibm-q-unibw',
                group='external-student',
                project='qnlp', ibmq_api_token='7c4a348dd83600545d560523c09e0cb05ce80191c267fcce23eba6e11cb8af4b5d446ab4182525881e9e3195a383a1006f92af6a4084d35b05e27f8456321d80')

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save an IBM Quantum account.
QiskitRuntimeService.save_account(channel="ibm_quantum", token="7c4a348dd83600545d560523c09e0cb05ce80191c267fcce23eba6e11cb8af4b5d446ab4182525881e9e3195a383a1006f92af6a4084d35b05e27f8456321d80")

In [4]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.get_backend("ibm_washington")

In [5]:
service = QiskitRuntimeService()
program_inputs = {'iterations': 1}
options = {"backend_name": "ibmq_qasm_simulator"}
job = service.run(program_id="hello-world",
                options=options,
                inputs=program_inputs
                )
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

job id: cgul4co43kb0aavfnkv0
Hello, World!


In [6]:
"""A sample runtime program that submits random circuits for user-specified iterations."""

import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit


def prepare_circuits(backend):
    """Generate a random circuit.

    Args:
        backend: Backend used for transpilation.

    Returns:
        Generated circuit.
    """
    circuit = random_circuit(
        num_qubits=5, depth=4, measure=True, seed=random.randint(0, 1000)
    )
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop("iterations", 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    return "Hello, World!"

In [7]:
from qiskit_ibm_runtime import RuntimeEncoder, RuntimeDecoder
from qiskit_ibm_runtime.program import UserMessenger, ProgramBackend
user_messenger = UserMessenger()
main(backend=backend, user_messenger=user_messenger)

RuntimeError: IBMBackend.run() is not supported in the Qiskit Runtime environment.